# Recommeneation Algorithm

**History**

In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
import os
import pandas as pd
import numpy as np
from py2neo import Graph, Node, Relationship, Subgraph, NodeMatcher
from graphdatascience import GraphDataScience
from tqdm import tqdm
import json
import joblib
pd.set_option('display.max_rows', 100)

In [2]:
#neo4j DB info
#7473/tcp, 0.0.0.0:7471->7474/tcp, 0.0.0.0:7681->7687/tcp
db_info = {
    #'uri' : "bolt://10.109.6.14:7681",
    'uri' : "bolt://localhost:7681",
    'user' : "neo4j",
    'password' : "admin",
    }
graph_db = Graph(db_info['uri'], user=db_info['user'], password=db_info['password'])
gds = GraphDataScience(db_info['uri'], auth=(db_info['user'], db_info['password']))


## **協同過濾 (User-based)**
- 選擇user: userid:debby
- 透過已預先運算的similar relationship, 篩選top k`相似user`
- `相似user`喜歡的文章計次排序

In [141]:
def collaborative_filtering(userid = 'debby', recom_qty=20):
    #喜歡的測試資料
    cypher = f'''match (n:User {{userid:'{userid}'}})-[:LIKE_TEST]->(d:Doc) return d.titleno'''
    cursor = graph_db.run(cypher)
    df = cursor.to_data_frame()
    ans_df = df['d.titleno'].value_counts().to_frame().reset_index()
    ans_df.columns = ['titleno','cnt']
    test_titleno = ans_df['titleno'].values
    
    #喜歡的訓練資料
    cypher = f'''match (n:User {{userid:'{userid}'}})-[:LIKE]->(d:Doc) return d.titleno'''
    cursor = graph_db.run(cypher)
    df = cursor.to_data_frame()
    ans_df = df['d.titleno'].value_counts().to_frame().reset_index()
    ans_df.columns = ['titleno','cnt']
    ans_titleno = ans_df['titleno'].values

    #有看過的
    cypher = f'''match (n:User {{userid:'{userid}'}})-[:DONT_LIKE]->(d:Doc) return d.titleno'''
    cursor = graph_db.run(cypher)
    df = cursor.to_data_frame()
    valid_df = df['d.titleno'].value_counts().to_frame().reset_index()
    valid_df.columns = ['titleno','cnt']
    valid_titleno = valid_df['titleno'].values

    #推薦
    cypher = f'''match (n:User {{userid:'{userid}'}})-[:SIMILAR_USER]->(n2)-[:LIKE]->(d:Doc) return d.titleno'''
    cursor = graph_db.run(cypher)
    df = cursor.to_data_frame()
    recomm_df = df['d.titleno'].value_counts().to_frame().reset_index()
    recomm_df.columns = ['titleno','cnt']
    #all_recomm = recomm_df['titleno'].values
    all_recomm = recomm_df['titleno'].values
    recomm_df = recomm_df[recomm_df['cnt']>1]
    recomm_titleno = recomm_df[:recom_qty]['titleno'].values
    #print(userid,len(recomm_titleno))
    #計分
    ret_dict = precision_score(valid_titleno, ans_titleno, test_titleno, recomm_titleno, all_recomm)
    return ret_dict
    
def precision_score(valid_titleno, ans_titleno, test_titleno, recomm_titleno, all_recomm):
    #-- prcision --
    ret_dict = {}
    pass_cnt = 0
    fail_cnt = 0
    invalid_cnt = 0
    training_cnt = 0
    for r in recomm_titleno:
        if r in test_titleno:
            pass_cnt+=1
            continue
        if r in ans_titleno:
            training_cnt+=1
            continue            
        if (r not in test_titleno) & (r not in valid_titleno):
            invalid_cnt+=1
            continue
        if (r not in test_titleno) & (r in valid_titleno):
            #print('FAIL',r)
            fail_cnt+=1
            continue        
        print('!!!!!!!!其他不預期狀況~~~~~~',r)
    ret_dict['pass']=pass_cnt
    ret_dict['precision_fail']=fail_cnt
    ret_dict['invalid (沒看過數量)']=invalid_cnt
    ret_dict['training (預測的是訓練資料)']=training_cnt    
    ret_dict['Precision'] = pass_cnt/(pass_cnt+fail_cnt) if (pass_cnt+fail_cnt)>0 else '--' 
    
    #-- recall --
    test_cnt = len(test_titleno)
    ret_dict['test data (測試資料數量)'] = test_cnt
    ret_dict['Recall'] = recall_passCnt/test_cnt
    
    if True:
        rank_list = []
        for t in test_titleno:
            if t in all_recomm:
                rank = np.argwhere(all_recomm==t)[0][0]
                rank_list.append(rank)
            else:
                rank_list.append(-1)
        ret_dict['rank']=rank_list
    return ret_dict


In [143]:
#prediction & 計算precision

cypher = f'''match(n:User) return n.userid'''
cursor = graph_db.run(cypher)
df = cursor.to_data_frame()
userid_list = df['n.userid'].values
ret_list = []
for useid in userid_list:
    #precision, recall, pass_cnt, fail_cnt, invalid_cnt, training_cnt, recall_passCnt, test_cnt = collaborative_filtering(useid, recom_qty=25)
    ret_dict = collaborative_filtering(useid, recom_qty=25)
    ret_dict['userid']=useid
    ret_list.append(ret_dict)
    #ret_list.append([useid, precision, pass_cnt, fail_cnt, invalid_cnt, training_cnt, recall, recall_passCnt, test_cnt])
    
precision_df = pd.DataFrame(ret_list)
#precision_df.columns=['useid','precision','pass_cnt','fail_cnt','沒看過數量','訓練資料','recall','recall_passCnt','test_cnt']
precision_all = precision_df['pass'].sum()/(precision_df['precision_fail'].sum()+precision_df['pass'].sum())    
print('precision:', precision_all)
recall_all = precision_df['pass'].sum()/precision_df['test data (測試資料數量)'].sum()    
print('recall:', recall_all)

precision_df[precision_df['Precision']!='--'].sort_values(by='Precision', ascending=False)


precision: 0.3235294117647059
recall: 0.0042985541227041815


,pass,precision_fail,invalid (沒看過數量),training (預測的是訓練資料),Precision,test data (測試資料數量),Recall,rank,userid
0,1,0,15,1,1.0,19,0.0,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, 14, -1, -...",509716024
70,1,0,23,1,1.0,17,0.0,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, 762, -1, ...",idaw03
10,1,0,24,0,1.0,42,0.0,"[-1, -1, 518, 607, -1, -1, -1, -1, -1, 872, -1...",JW
66,1,0,4,1,1.0,76,0.0,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...",andrew0717
18,1,0,20,4,1.0,141,0.0,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...",kenny
25,1,0,6,2,1.0,85,0.0,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...",Test
44,1,0,8,0,1.0,66,0.0,"[-1, 108, -1, -1, -1, -1, -1, -1, -1, -1, -1, ...",fxn
15,2,1,22,0,0.666667,103,0.0,"[-1, -1, -1, -1, -1, -1, 901, -1, -1, -1, -1, ...",bird
26,1,1,15,0,0.5,10,0.0,"[-1, -1, 15, -1, -1, -1, -1, -1, -1, -1]",finley
8,1,3,20,0,0.25,46,0.0,"[-1, -1, -1, 82, 84, 23, -1, -1, -1, -1, -1, -...",hhug_y


## **協同過濾 (Item-based)**
- 選擇user
- 篩選出user喜歡的document
- 透過已預先運算的similar relationship, 篩選top k`相似document`
- `相似document`計次排序

In [198]:
def collaborative_itembase(userid = 'debby', recom_qty=20):
    #喜歡的測試資料
    cypher = f'''match (n:User {{userid:'{userid}'}})-[:LIKE_TEST]->(d:Doc) return d.titleno'''
    cursor = graph_db.run(cypher)
    df = cursor.to_data_frame()
    ans_df = df['d.titleno'].value_counts().to_frame().reset_index()
    ans_df.columns = ['titleno','cnt']
    test_titleno = ans_df['titleno'].values
    
    #喜歡的訓練資料
    cypher = f'''match (n:User {{userid:'{userid}'}})-[:LIKE]->(d:Doc) return d.titleno'''
    cursor = graph_db.run(cypher)
    df = cursor.to_data_frame()
    ans_df = df['d.titleno'].value_counts().to_frame().reset_index()
    ans_df.columns = ['titleno','cnt']
    ans_titleno = ans_df['titleno'].values

    #有看過的
    cypher = f'''match (n:User {{userid:'{userid}'}})-[:DONT_LIKE]->(d:Doc) return d.titleno'''
    cursor = graph_db.run(cypher)
    df = cursor.to_data_frame()
    valid_df = df['d.titleno'].value_counts().to_frame().reset_index()
    valid_df.columns = ['titleno','cnt']
    valid_titleno = valid_df['titleno'].values

    #推薦
    cypher = f'''MATCH (u:User {{userid:'{userid}'}})-[:LIKE]->(d:Doc)-[r:SIMILAR_DOC]->(d2:Doc) RETURN d2.titleno as titleno, r.score as score ORDER BY score DESC  LIMIT 100
'''
    cursor = graph_db.run(cypher)
    df = cursor.to_data_frame()
    if False:
        recomm_df = df['titleno'].value_counts().to_frame().reset_index()
        recomm_df.columns = ['titleno','cnt']
        all_recomm = recomm_df['titleno'].values
        recomm_df = recomm_df[recomm_df['cnt']>1]
        recomm_titleno = recomm_df['titleno'].values[:recom_qty]
    else:
        df['cnt']=1
        recomm_df = df.groupby('titleno').agg({'cnt':'count','score':'mean'})
        recomm_df.sort_values(['cnt','score'], inplace=True)
        recomm_df=recomm_df.reset_index()
        all_recomm = recomm_df['titleno'].values
        recomm_titleno = recomm_df['titleno'].values[:recom_qty]
    
    #計分
    ret_dict = precision_score(valid_titleno, ans_titleno, test_titleno, recomm_titleno, all_recomm)
    return ret_dict
    

In [199]:
#prediction & 計算precision

cypher = f'''match(n:User) return n.userid'''
cursor = graph_db.run(cypher)
df = cursor.to_data_frame()
userid_list = df['n.userid'].values
ret_list = []
for useid in userid_list:
    ret_dict = collaborative_itembase(useid, recom_qty=20)
    ret_dict['userid']=useid
    ret_list.append(ret_dict)
    
precision_df = pd.DataFrame(ret_list)
#precision_df.columns=['useid','precision','pass_cnt','fail_cnt','沒看過數量']
precision_all = precision_df['pass'].sum()/(precision_df['precision_fail'].sum()+precision_df['pass'].sum())    
print('precision:', precision_all)
recall_all = precision_df['pass'].sum()/precision_df['test data (測試資料數量)'].sum()    
print('recall:', recall_all)

precision_df[precision_df['Precision']!='--'].sort_values(by='Precision', ascending=False)

precision: 0.12195121951219512
recall: 0.0019538882375928096


,pass,precision_fail,invalid (沒看過數量),training (預測的是訓練資料),Precision,test data (測試資料數量),Recall,rank,userid
27,1,0,17,2,1.0,196,0.0,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...",p12345
31,1,0,19,0,1.0,98,0.0,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...",QQ
15,1,0,19,0,1.0,103,0.0,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...",bird
18,1,0,19,0,1.0,141,0.0,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...",kenny
22,1,1,18,0,0.5,85,0.0,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...",pinch
65,0,1,19,0,0.0,16,0.0,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...",idawang003
61,0,1,19,0,0.0,10,0.0,"[53, -1, -1, -1, -1, -1, -1, -1, -1, -1]",W0106957
56,0,1,19,0,0.0,15,0.0,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...",joe
49,0,1,19,0,0.0,25,0.0,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...",eightno8
43,0,3,17,0,0.0,33,0.0,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...",tengigabytes
